# adabound.py実行確認
- tfgpu_py36_v3 環境


- Github: https://github.com/titu1994/keras-adabound


- https://qiita.com/Phoeboooo/items/f610affdcaaae0a28f34
    - Adamに学習率の上限と下限を動的に加えたものをAdaBound
    - AMSGradに学習率の上限と下限を動的に加えたものをAMSBound
    - どちらの手法も最初はAdamのように動き、後半からSGDのように動く
    - Adamの良さである初期の学習の速さとSGDの良さである汎化能力を両立した最適化手法

In [1]:
# adabound.py
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Optimizer


class AdaBound(Optimizer):
    """AdaBound optimizer.

    Default parameters follow those provided in the original paper.

    # Arguments
        lr: float >= 0. Learning rate.
        final_lr: float >= 0. Final learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        gamma: float >= 0. Convergence speed of the bound function.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsbound: boolean. Whether to apply the AMSBound variant of this
            algorithm.

    # References
        - [Adaptive Gradient Methods with Dynamic Bound of Learning Rate]
          (https://openreview.net/forum?id=Bkg3g2R9FX)
        - [Adam - A Method for Stochastic Optimization]
          (https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond]
          (https://openreview.net/forum?id=ryQu7f-RZ)
    """

    def __init__(self, lr=0.001, final_lr=0.1, beta_1=0.9, beta_2=0.999, gamma=1e-3,
                 epsilon=None, decay=0., amsbound=False, weight_decay=0.0, **kwargs):
        super(AdaBound, self).__init__(**kwargs)

        if not 0. <= gamma <= 1.:
            raise ValueError("Invalid `gamma` parameter. Must lie in [0, 1] range.")

        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')

        self.final_lr = final_lr
        self.gamma = gamma

        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsbound = amsbound

        self.weight_decay = float(weight_decay)
        self.base_lr = float(lr)

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        # Applies bounds on actual learning rate
        step_size = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                          (1. - K.pow(self.beta_1, t)))

        final_lr = self.final_lr * lr / self.base_lr
        lower_bound = final_lr * (1. - 1. / (self.gamma * t + 1.))
        upper_bound = final_lr * (1. + 1. / (self.gamma * t))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsbound:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            # apply weight decay
            if self.weight_decay != 0.:
                g += self.weight_decay * K.stop_gradient(p)

            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            if self.amsbound:
                vhat_t = K.maximum(vhat, v_t)
                denom = (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                denom = (K.sqrt(v_t) + self.epsilon)

            # Compute the bounds
            step_size_p = step_size * K.ones_like(denom)
            step_size_p_bound = step_size_p / denom
            # TODO: Replace with K.clip after releast of Keras > 2.2.4
            bounded_lr_t = m_t * tf.clip_by_value(step_size_p_bound,
                                                  lower_bound,
                                                  upper_bound)

            p_t = p - bounded_lr_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'final_lr': float(self.final_lr),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'gamma': float(self.gamma),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'weight_decay': self.weight_decay,
                  'amsbound': self.amsbound}
        base_config = super(AdaBound, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


Using TensorFlow backend.


In [6]:
# adabound parameters
adabound_final_lr = 0.1
adabound_gamma = 1e-3
weight_decay = 5e-4
amsbound = True

optimizer=AdaBound(lr=1.0
                   , final_lr=adabound_final_lr
                   , gamma=adabound_gamma
                   , weight_decay=weight_decay
                   , amsbound=amsbound)

optimizer

In [9]:
import keras
from keras.datasets import cifar10
from resnet import ResNet18, ResNet34

# Training parameters
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]
print(input_shape)

# Normalize data.
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


model = ResNet18(input_shape=input_shape, depth=18)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

(32, 32, 3)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 32, 32, 64)   1792        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 32, 32, 64)   256         conv2d_61[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 32, 32, 64)   0           batch_normalization_52[0][0]     
_________________________________________________________________________________________________

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
50000/50000 [==============================] - 55s 1ms/step - loss: 18.6587 - acc: 0.0998 - val_loss: 18.1087 - val_acc: 0.1000
Epoch 2/200
50000/50000 [==============================] - 48s 954us/step - loss: 17.8309 - acc: 0.1000 - val_loss: 17.6013 - val_acc: 0.1000
Epoch 3/200
50000/50000 [==============================] - 48s 955us/step - loss: 17.4299 - acc: 0.1000 - val_loss: 17.2746 - val_acc: 0.1000
Epoch 4/200
50000/50000 [==============================] - 47s 949us/step - loss: 17.1449 - acc: 0.1000 - val_loss: 17.0235 - val_acc: 0.1000
Epoch 5/200
50000/50000 [==============================] - 47s 949us/step - loss: 16.9172 - acc: 0.1000 - val_loss: 16.8160 - val_acc: 0.1000
Epoch 6/200
50000/50000 [==============================] - 48s 952us/step - loss: 16.7251 - acc: 0.1000 - val_loss: 16.6377 - val_acc: 0.1000
Epoch 7/200
50000/50000 [==============================] - 47s 950us/step - loss: 16.5580 - acc: 0.1

50000/50000 [==============================] - 47s 946us/step - loss: 14.5988 - acc: 0.1000 - val_loss: 14.5961 - val_acc: 0.1000
Epoch 59/200
50000/50000 [==============================] - 47s 946us/step - loss: 14.5936 - acc: 0.1000 - val_loss: 14.5911 - val_acc: 0.1000
Epoch 60/200
50000/50000 [==============================] - 47s 946us/step - loss: 14.5888 - acc: 0.1000 - val_loss: 14.5864 - val_acc: 0.1000
Epoch 61/200
50000/50000 [==============================] - 47s 946us/step - loss: 14.5842 - acc: 0.1000 - val_loss: 14.5820 - val_acc: 0.1000
Epoch 62/200
50000/50000 [==============================] - 47s 945us/step - loss: 14.5799 - acc: 0.1000 - val_loss: 14.5778 - val_acc: 0.1000
Epoch 63/200
50000/50000 [==============================] - 47s 945us/step - loss: 14.5758 - acc: 0.1000 - val_loss: 14.5738 - val_acc: 0.1000
Epoch 64/200
50000/50000 [==============================] - 47s 948us/step - loss: 14.5719 - acc: 0.1000 - val_loss: 14.5701 - val_acc: 0.1000
Epoch 65/200

Epoch 173/200
50000/50000 [==============================] - 47s 947us/step - loss: 2.3031 - acc: 0.0982 - val_loss: 2.3029 - val_acc: 0.1000
Epoch 174/200
50000/50000 [==============================] - 47s 948us/step - loss: 2.3031 - acc: 0.0958 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 175/200
50000/50000 [==============================] - 47s 946us/step - loss: 2.3030 - acc: 0.0984 - val_loss: 2.3029 - val_acc: 0.1000
Epoch 176/200
50000/50000 [==============================] - 47s 946us/step - loss: 2.3030 - acc: 0.1001 - val_loss: 2.3029 - val_acc: 0.1000
Epoch 177/200
50000/50000 [==============================] - 47s 945us/step - loss: 2.3030 - acc: 0.0987 - val_loss: 2.3029 - val_acc: 0.1000
Epoch 178/200
50000/50000 [==============================] - 47s 946us/step - loss: 2.3030 - acc: 0.1001 - val_loss: 2.3029 - val_acc: 0.1000
Epoch 179/200
50000/50000 [==============================] - 47s 946us/step - loss: 2.3031 - acc: 0.0981 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 

In [9]:
import os, sys
sys.path.append(r'C:\Users\shingo\Git\keras-adabound')
from adabound import AdaBound
#print(sys.modules.keys())
print('adabound' in sys.modules.keys())

# adabound parameters
adabound_final_lr = 0.1
adabound_gamma = 1e-3
weight_decay = 5e-4
amsbound = True

optimizer=AdaBound(lr=1.0
                   , final_lr=adabound_final_lr
                   , gamma=adabound_gamma
                   , weight_decay=weight_decay
                   , amsbound=amsbound)

optimizer

True


In [1]:
# モジュールimport
import os, sys
sys.path.append(r'C:\Users\shingo\jupyter_notebook\tfgpu_py36_work\02_keras_py')
from model import define_model

define_model.get_adabound()

Using TensorFlow backend.


define_model.py: loaded as module file


In [1]:
# モジュールimport
import os, sys
sys.path.append(r'C:\Users\shingo\jupyter_notebook\tfgpu_py36_work\02_keras_py')
from model import define_model

define_model.get_optimizers(choice_optim='adabound')

Using TensorFlow backend.


define_model.py: loaded as module file
---- choice_optim = adabound ----
adabound_lr adabound_decay beta_1 beta_2, amsbound = 0.001 0.0 0.9 0.999 False
